In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.metrics import mean_squared_error

In [2]:
pune_tmax_train = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_train_clean.csv')
Nex_train = pd.read_csv(r'D:\IITM\prepared data\Nex_train_ready.csv')

pune_tmax_test = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_pune_test.csv')
Nex_test = pd.read_csv(r'D:\IITM\prepared data\Nex_test_ready.csv')

#using pune tmax values as labels 
tmax_labels = pd.DataFrame(pune_tmax_train,columns=["tmax"])
tmax_labels.head() # .head() function gives out first 5 rows of the datset 

,tmax
0,28.4
1,29.4
2,29.1
3,28.3
4,28.5


In [3]:
Nex_train = Nex_train.drop(['nex_tmean'],axis=1)
Nex_test = Nex_test.drop(['nex_tmean'],axis=1)

In [4]:
Nex_train.head()

,nex_tmax,nex_tmin
0,30.1280,10.7089
1,30.4229,10.5904
2,30.4062,10.6063
3,30.5158,11.8068
4,30.4171,13.2873


In [5]:
Nex_test.head()

,nex_tmax,nex_tmin
0,30.0709,14.5580
1,30.3630,14.8125
2,30.2909,14.9060
3,30.3834,14.2860
4,30.0295,13.7385


In [6]:
test_labels = pd.DataFrame(pune_tmax_test,columns=["tmax"])
test_labels.head()

,tmax
0,29.7
1,30.0
2,28.2
3,28.3
4,29.6


In [7]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(Nex_train)
y_train = sc_y.fit_transform(tmax_labels)
X_test = sc_X.fit_transform(Nex_test)
y_test = sc_y.fit_transform(test_labels)

In [9]:
from sklearn import svm
tmax_svr = svm.SVR()
tmax_svr.fit(X_train , y_train)

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR()

In [10]:
train_pred_svr = tmax_svr.predict(X_train)
train_pred_svr = sc_y.inverse_transform(train_pred_svr)
#calculating RMSE over train set for SVR

Nex_svr_mse_train = mean_squared_error(tmax_labels, train_pred_svr)
Nex_svr_rmse_train = np.sqrt(Nex_svr_mse_train)
print("RMSE for Support Vector Regression over train set = ")
Nex_svr_rmse_train

RMSE for Support Vector Regression over train set = 


1.6096252788826082

In [11]:
test_pred_svr = tmax_svr.predict(X_test)
test_pred_svr = sc_y.inverse_transform(test_pred_svr)
#calculating RMSE over train set for SVR
from sklearn.metrics import mean_squared_error
Nex_svr_mse_test = mean_squared_error(test_labels, test_pred_svr)
Nex_svr_rmse_test = np.sqrt(Nex_svr_mse_test)
print("RMSE for Support Vector Regression over test set = ")
Nex_svr_rmse_test

RMSE for Support Vector Regression over test set = 


1.648475788931011

In [10]:
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit

In [14]:
svr_para = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto'],'C':[0.1,1,5]}
svr_tune = GridSearchCV(svm.SVR(), param_grid = svr_para, n_jobs=-1)
svr_tune.fit(X_train, y_train)
print(svr_tune.score(X_train, y_train))
print(svr_tune.best_params_)

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.6896462759645559
{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}


In [15]:
svr_tuned = svm.SVR(C= 0.1,gamma='auto',kernel='rbf')
svr_tuned.fit(X_train , y_train)

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR(C=0.1, gamma='auto')

In [16]:
svr_tuned_train_pred = svr_tuned.predict(X_train)
svr_tuned_train_pred = sc_y.inverse_transform(svr_tuned_train_pred)
#calculating RMSE over train set for SVR

Nex_svr_mse_train = mean_squared_error(tmax_labels, svr_tuned_train_pred)
Nex_svr_rmse_train = np.sqrt(Nex_svr_mse_train)
print("RMSE for Support Vector Regression over train set = ")
Nex_svr_rmse_train

RMSE for Support Vector Regression over train set = 


1.6177263599021212

In [17]:
svr_tuned_test_pred = svr_tuned.predict(X_test)
svr_tuned_test_pred = sc_y.inverse_transform(svr_tuned_test_pred)
#calculating RMSE over train set for SVR
from sklearn.metrics import mean_squared_error
Nex_svr_mse_test = mean_squared_error(test_labels, svr_tuned_test_pred)
Nex_svr_rmse_test = np.sqrt(Nex_svr_mse_test)
print("RMSE for Support Vector Regression over test set = ")
Nex_svr_rmse_test

RMSE for Support Vector Regression over test set = 


1.638664964317867

In [18]:
from sklearn.neural_network import MLPRegressor

In [19]:
parameters = {'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs'], 'max_iter': [700,800,900]}
clf = GridSearchCV(MLPRegressor(), parameters, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.6967208724948044
{'activation': 'logistic', 'alpha': 1e-06, 'hidden_layer_sizes': 14, 'max_iter': 800, 'solver': 'lbfgs'}


In [20]:
mlp_reg = MLPRegressor(activation='logistic', alpha= 1e-06, hidden_layer_sizes= 14, max_iter= 800, solver='lbfgs')
mlp_reg.fit(X_train, y_train)
train_preds = mlp_reg.predict(X_train)
test_preds = mlp_reg.predict(X_test)
train_pred_mlp = sc_y.inverse_transform(train_preds)
test_pred_mlp = sc_y.inverse_transform(test_preds)

mlp_reg_mse_train = mean_squared_error(test_pred_mlp, test_labels)
mlp_reg_mse_test = mean_squared_error(train_pred_mlp, tmax_labels)
mlp_reg_rmse_train = np.sqrt(mlp_reg_mse_train)
mlp_reg_rmse_test = np.sqrt(mlp_reg_mse_test)
print("RMSE of the MLP with Grid Search on trainset: ", mlp_reg_rmse_train, "\n\n")
print("RMSE of the MLP with Grid Search on testset: ", mlp_reg_rmse_test, "\n\n")

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RMSE of the MLP with Grid Search on trainset:  1.6449566612263222 


RMSE of the MLP with Grid Search on testset:  1.5871901582779402 




c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
estimator = RandomForestRegressor()
param_grid = { 
            "n_estimators"      : [400, 500, 600, 700, 800],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }

grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

grid.fit(X_train, y_train)
print(grid.best_score_) 
print(grid.best_params_)

c:\users\narayan\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


0.6470082356354998
{'bootstrap': True, 'max_features': 'log2', 'min_samples_split': 8, 'n_estimators': 600}


In [12]:
Nex_rfr_reg = RandomForestRegressor(bootstrap=True, max_features= 'log2', min_samples_split= 8, n_estimators=600).fit(X_train, y_train)

C:\Users\Narayan\AppData\Local\Temp/ipykernel_10072/3345772182.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Nex_rfr_reg = RandomForestRegressor(bootstrap=True, max_features= 'log2', min_samples_split= 8, n_estimators=600).fit(X_train, y_train)


In [13]:
Nex_rfr_train_preds = Nex_rfr_reg.predict(X_train)
Nex_rfr_test_preds = Nex_rfr_reg.predict(X_test)
train_pred_rfr = sc_y.inverse_transform(Nex_rfr_train_preds)
test_pred_rfr = sc_y.inverse_transform(Nex_rfr_test_preds)

Nex_rfr_reg_mse_train = mean_squared_error(train_pred_rfr, tmax_labels)
Nex_rfr_reg_mse_test = mean_squared_error(test_pred_rfr, test_labels)
Nex_rfr_reg_rmse_train = np.sqrt(Nex_rfr_reg_mse_train)
Nex_rfr_reg_rmse_test = np.sqrt(Nex_rfr_reg_mse_test)
print("RMSE of the Random Forest Regression with scaling trainset: ", Nex_rfr_reg_rmse_train,"\n")
print("RMSE of the Random Forest Regression with scaling testset: ", Nex_rfr_reg_rmse_test, "\n")

RMSE of the Random Forest Regression with scaling trainset:  1.0296881075644033 

RMSE of the Random Forest Regression with scaling testset:  1.7318002483285497 

